In [1]:
import requests, json, time

# files
ChampPickedCounterFile = open("ChampPickedCounter.txt", "w")
champWinRateFile = open("champWinRate.txt", "w")
ChampWinListFile = open("ChampWinList.txt", "w")
ChampLoseListCounterFile = open("ChampLoseListCounter.txt", "w")
GamesInfoFile = open("GamesInfo.txt", "w")

# variables 
Patch = "11.24.1"
MinimumGamesPlayed = 100
Champion = "Evelynn"

In [2]:
# Recupere la list des chall
api_key = "RGAPI-034a4ddc-4862-4229-80cf-face74a491d9"
GetChallPlayers = "https://euw1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5"+ "?api_key=" + api_key
response1 = requests.get(GetChallPlayers).json()

In [3]:
# recupere le puuid des chall
print (time.strftime("%H : %M : %S"))
info = (response1['entries'])
playersPuuid = []
playersName = []
for i in range(299):
        GetPlayersPuuid = "https://euw1.api.riotgames.com/lol/summoner/v4/summoners/"+info[i]['summonerId']+ "?api_key=" + api_key
        response2 = requests.get(GetPlayersPuuid).json()
        playersPuuid.append(response2["puuid"])
        time.sleep(2)
print("end")
print (time.strftime("%H : %M : %S"))
# recupere les 20 dernieres games de chaque joueur depuis une certaine date
print (time.strftime("%H : %M : %S"))
PlayersGames = []
date = "1639029600"
for i in range(299):
        GetPlayersGames = "https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/" + playersPuuid[i] + "/ids?startTime=" + date +"&queue=420&type=ranked&start=0&count=20&api_key=" + api_key
        response2 = requests.get(GetPlayersGames).json()
        PlayersGames = PlayersGames + response2
        time.sleep(2)
print("end")
print (time.strftime("%H : %M : %S"))
PlayersGames.sort()
# verifie que la liste des games a ete trie
print(len(PlayersGames))
PlayersGames = list(set(PlayersGames))
seen = set()
result = []
for item in PlayersGames:
    if item not in seen:
        seen.add(item)
        result.append(item)
print(len(result))
# recupere les infos de chaque game
print (time.strftime("%H : %M : %S"))
GamesInfo = []
PlayersGamesRange = len(PlayersGames) - 1
for i in range(PlayersGamesRange):
    GetGamesInfo = "https://europe.api.riotgames.com/lol/match/v5/matches/" + PlayersGames[i] + "?api_key=" + api_key
    response3 = requests.get(GetGamesInfo).json()
    GamesInfo.append(response3)
    time.sleep(2)
print("end")
print (time.strftime("%H : %M : %S"))


11 : 46 : 02
end
11 : 56 : 40
11 : 56 : 40
end
12 : 07 : 16
2443
1638
12 : 07 : 16
end
13 : 06 : 44


In [13]:
#nb de games par champions
# recupere la liste des champions joue sur toutes les games dispo, et on tri par ordre alphabet
from operator import itemgetter
champsList = []
count = len(GamesInfo) - 1
for j in range (count):
    for i in range (10):
        champsList.append(GamesInfo[j]["info"]["participants"][i]["championName"])
# compte le nombre d apparition d un champion
ChampWinList = [[x,champsList.count(x)] for x in set(champsList)]
ChampPickedCounter = [[x,champsList.count(x)] for x in set(champsList)]
# pour trier par nb de games : , key=itemgetter(1), reverse=True
ChampPickedCounter = sorted(ChampPickedCounter)
ChampWinList = sorted(ChampWinList)
#nb de loose par champions
# recupere la liste des champions joue sur toutes les games dispo, et on tri par ordre alphabet
from operator import itemgetter
ChampLoseList = []
count = len(GamesInfo) - 1
for j in range (count):
    for i in range (10):
        if GamesInfo[j]["info"]["participants"][i]["win"] == False:
            ChampLoseList.append(GamesInfo[j]["info"]["participants"][i]["championName"])
# compte le nombre d apparition d un champion
ChampLoseListCounter = [[x,ChampLoseList.count(x)] for x in set(ChampLoseList)]
# pour trier par nb de games : , key=itemgetter(1), reverse=True
ChampLoseListCounter = sorted(ChampLoseListCounter)
#nb de games loose par champions
import copy
champWinRate = []
countLose =  len(ChampLoseListCounter) - 1
countPick = len(ChampPickedCounter) - 1
# calcul du winrate d un champion 
for i in range (countPick):
    for j in range (countLose):
        if ChampWinList[i][0] == ChampLoseListCounter[j][0]:
            ChampWinList[i][1] = ChampWinList[i][1] - ChampLoseListCounter[j][1]
            winrate = round(ChampWinList[i][1] * 100 / ChampPickedCounter[i][1], 2)
            if ChampPickedCounter[i][1] > MinimumGamesPlayed:
                champWinRate.append([ChampWinList[i][0], winrate])
champWinRate = sorted(champWinRate, key=lambda x: x[1], reverse=True)

In [14]:
# on affiche toutes nos donnees

print("------------Picks-----------------")
print(sorted(ChampPickedCounter, key=lambda x: x[1], reverse=True))
print("-------------Win rate----------------")
print(champWinRate)
print("------------win nb-----------------")
print(ChampWinList)
print("-------------lose nb----------------")
print(ChampLoseListCounter)

------------Picks-----------------
[['LeeSin', 552], ['Viego', 452], ['Jinx', 415], ['Jhin', 398], ['Kaisa', 368], ['Lux', 352], ['Qiyana', 344], ['Viktor', 335], ['Yone', 332], ['Lulu', 323], ['Jayce', 318], ['Caitlyn', 295], ['Ezreal', 293], ['Akshan', 288], ['Zed', 263], ['Karma', 254], ['Akali', 247], ['Vex', 242], ['Rakan', 241], ['Graves', 240], ['Talon', 234], ['Irelia', 232], ['Thresh', 227], ['Lucian', 217], ['Syndra', 217], ['Corki', 211], ['Fiora', 210], ['Nami', 207], ['Janna', 200], ['Sylas', 200], ['Yasuo', 195], ['Nidalee', 193], ['Leona', 181], ['Gragas', 178], ['Leblanc', 173], ['Ekko', 170], ['XinZhao', 165], ['Nautilus', 160], ['Pyke', 158], ['Vayne', 148], ['TwistedFate', 147], ['Aphelios', 141], ['Draven', 141], ['Kayn', 132], ['Singed', 119], ['Camille', 113], ['Bard', 112], ['Twitch', 112], ['Sona', 109], ['Zoe', 109], ['Jax', 108], ['Kassadin', 102], ['Xerath', 102], ['Ashe', 96], ['Riven', 91], ['Orianna', 89], ['Tryndamere', 88], ['Yuumi', 88], ['Gwen', 86], [

In [15]:
# Calcul du nombre d'occurrences des items pour un champion donne
ItemList = []
WinOrNot = True
for e in range(count):
    for i in range(10):
        for j in range(6):
            if GamesInfo[e]["info"]["participants"][i]["championName"] == Champion:
#                 est ce que c est une game win ou non (peut etre mis en commit)
#                 if GamesInfo[e]["info"]["participants"][i]["win"] == TrueOrFalse:
                    ItemList.append(GamesInfo[e]["info"]["participants"][i]["item" + str(j)])
print(ItemList)
ItemListcounter = [[x,ItemList.count(x)] for x in set(ItemList)]
ItemListcounter = sorted(ItemListcounter, key=lambda x: x[1], reverse=True)

# on change l'alias des items pour leur nom
GetItemInfo = requests.get('https://ddragon.leagueoflegends.com/cdn/'+ Patch +'/data/en_US/item.json').json()
for i in range(len(ItemListcounter)):
    if ItemListcounter[i][0] != 0:
        number = str(ItemListcounter[i][0])
        ItemListcounter[i][0] = GetItemInfo["data"][str(number)]["name"]
    if ItemListcounter[i][0] == 0:
        ItemListcounter[i][0] = "null"
        
print(ItemListcounter)

[3020, 3152, 4645, 1082, 0, 3135, 3152, 1026, 3089, 0, 3041, 3020, 3089, 3041, 3152, 3020, 3100, 1026, 3152, 1058, 3020, 1082, 3102, 1058, 3152, 2031, 3020, 3089, 3113, 3041, 3152, 3089, 3102, 3041, 3135, 3020, 3152, 2031, 3020, 1082, 1058, 3145, 3152, 3089, 3041, 3135, 3020, 3108, 3152, 3100, 3089, 0, 3102, 3020, 3020, 3135, 3152, 3041, 1058, 1058, 1052, 3152, 3041, 3020, 0, 0, 2031, 3152, 3041, 0, 3020, 0, 3020, 2031, 3041, 3089, 0, 3152, 2422, 2031, 3152, 3041, 1052, 0, 3152, 4645, 4630, 3020, 3089, 0, 3152, 3102, 3041, 3020, 0, 0, 3152, 3020, 3089, 3145, 1082, 0, 3102, 3100, 3152, 3089, 3020, 3135, 3020, 1026, 3152, 3041, 3089, 0, 4645, 3135, 3152, 3089, 4629, 3020, 3152, 3041, 3102, 3020, 4630, 3089, 3020, 4645, 3152, 3041, 1026, 1052, 1058, 2031, 3152, 3041, 3020, 4645, 3152, 3100, 4629, 3102, 3089, 3041, 3152, 1026, 3020, 1082, 3102, 4630, 2055, 2031, 3152, 3041, 3089, 3020, 3020, 3135, 3152, 3041, 0, 1028, 3020, 3057, 3152, 1082, 1026, 3113, 3152, 3020, 3041, 3089, 3135, 1058, 

In [16]:
# calcul du winrate par rapport au premier objectif
GamesFirstObjWin = 0
Obj = "inhibitor"
GameWin = True
FirstObj = True
for i in range(len(GamesInfo)):
    for e in range(2):
        if GamesInfo[i]["info"]["teams"][e]["win"] == GameWin and GamesInfo[i]["info"]["teams"][e]["objectives"][Obj]["first"] == FirstObj:
            GamesFirstObjWin = GamesFirstObjWin + 1
GamesFirstObjWinRate = GamesFirstObjWin * 100  / len(GamesInfo)
GamesFirstObjWinRate = round(GamesFirstObjWinRate, 2)
print("Winrate with",Obj,"killed first",GamesFirstObjWinRate)

Winrate with inhibitor killed first 73.61


In [17]:
# creation d un dictionaire avec la key et le nom du champion
bannedchamp = []
GetChampName = requests.get('https://ddragon.leagueoflegends.com/cdn/'+ Patch +'/data/en_US/champion.json').json()
ChampNameList = []
for k in GetChampName["data"]:
    ChampNameList.append(k)
ChampListKey = []
for i in range(len(ChampNameList)):
    ChampListKey.append([ChampNameList[i], GetChampName["data"][ChampNameList[i]]["key"]])

# calcul du nombre de ban d un champion
for f in range(len(GamesInfo)):
    for e in range(2):
        for i in range(5):
            if GamesInfo[f]["info"]["teams"][e]["bans"][i]["championId"] != -1:
                bannedchamp.append(GamesInfo[f]["info"]["teams"][e]["bans"][i]["championId"])
bannedchamp = [[x,bannedchamp.count(x)] for x in set(bannedchamp)]
bannedchamp = sorted(bannedchamp, key=lambda x: x[1], reverse=True)

# on change la key par le nom du champion
for i in range(len(bannedchamp)):
    for e in range(len(ChampListKey)):
        if str(bannedchamp[i][0]) == ChampListKey[e][1]:
            bannedchamp[i][0] = ChampListKey[e][0]
print("Most banned champ :", bannedchamp)

Most banned champ : [['Talon', 1232], ['Akshan', 964], ['Yuumi', 893], ['Kassadin', 797], ['Viktor', 657], ['Qiyana', 629], ['Irelia', 594], ['Draven', 492], ['Jax', 474], ['Yone', 463], ['Viego', 457], ['Fiora', 444], ['Lulu', 407], ['LeeSin', 404], ['Leblanc', 376], ['Zed', 332], ['Vex', 296], ['Tryndamere', 277], ['Evelynn', 275], ['Katarina', 250], ['Jinx', 221], ['Camille', 170], ['Rengar', 167], ['Thresh', 158], ['Jayce', 152], ['Lucian', 135], ['Akali', 132], ['Karthus', 128], ['Bard', 117], ['Yasuo', 112], ['TwistedFate', 111], ['Nami', 96], ['Lux', 93], ['Kayn', 91], ['Jhin', 90], ['Nidalee', 85], ['Vayne', 81], ['Singed', 79], ['Nunu', 75], ['Karma', 75], ['Hecarim', 74], ['Kaisa', 73], ['TahmKench', 72], ['Khazix', 71], ['Darius', 69], ['Corki', 67], ['Malphite', 65], ['Sona', 62], ['Leona', 61], ['Caitlyn', 58], ['Riven', 58], ['XinZhao', 55], ['Xerath', 52], ['Anivia', 49], ['Zilean', 46], ['Xayah', 45], ['Blitzcrank', 43], ['Sett', 43], ['Pyke', 42], ['Shaco', 40], ['Naut

In [18]:
WinBlueCount = 0

LoseBlueCount = 0

for i in range(len(GamesInfo)):
    if GamesInfo[i]["info"]["teams"][0]["win"] == True:
        WinBlueCount = WinBlueCount + 1
    elif GamesInfo[i]["info"]["teams"][0]["win"] == False:
        LoseBlueCount = LoseBlueCount + 1

print("Blue side win : ",WinBlueCount)

print("Blue side lose : ",LoseBlueCount)


Blue side win :  833
Blue side lose :  804


In [87]:
#Average game time 
GameTime = []
AverageTime = 0
for i in range(len(GamesInfo)):
    GameTime.append(GamesInfo[i]["info"]["gameDuration"])
AverageTime = round(sum(GameTime) / len(GameTime), 2)
AverageTimeInMinute = time.strftime("%M:%S", time.localtime(AverageTime))

# calcul du gain d exp et de gold des supports
SupportGoldEarned = []
SupportExp = []
AverageSupportGold = 0
AverageSupportExp = 0
for i in range(len(GamesInfo)):
    for e in range(10):
        if GamesInfo[i]["info"]["participants"][e]["role"] == "SUPPORT":
            SupportGoldEarned.append(GamesInfo[i]["info"]["participants"][e]["goldEarned"])
AverageSupportGold = sum(SupportGoldEarned) / len(SupportGoldEarned)
AverageSupportGold = round(AverageSupportGold, 2)
AverageSupportGoldMin = round((AverageSupportGold / AverageTime) * 60, 2)

for i in range(len(GamesInfo)):
    for e in range(10):
        if GamesInfo[i]["info"]["participants"][e]["role"] == "SUPPORT":
            SupportExp.append(GamesInfo[i]["info"]["participants"][e]["champExperience"])
AverageSupportExp = sum(SupportExp) / len(SupportExp)
AverageSupportExp = round(AverageSupportExp, 2)
AverageSupportExpMin = round((AverageSupportExp / AverageTime) * 60, 2)

# calcul du gain d exp et de gold des jungler
JunglerGoldEarned = []
JunglerExp = []
AverageJunglerExp = 0
AverageJunglerGold = 0
for i in range(len(GamesInfo)):
    for e in range(10):
        if GamesInfo[i]["info"]["participants"][e]["individualPosition"] == "JUNGLE":
            JunglerGoldEarned.append(GamesInfo[i]["info"]["participants"][e]["goldEarned"])
AverageJunglerGold = sum(JunglerGoldEarned) / len(JunglerGoldEarned)
AverageJunglerGold = round(AverageJunglerGold, 2)
AverageJunglerGoldMin = round((AverageJunglerGold / AverageTime) * 60, 2)

for i in range(len(GamesInfo)):
    for e in range(10):
        if GamesInfo[i]["info"]["participants"][e]["individualPosition"] == "JUNGLE":
            JunglerExp.append(GamesInfo[i]["info"]["participants"][e]["champExperience"])
AverageJunglerExp = sum(JunglerExp) / len(JunglerExp)
AverageJunglerExp = round(AverageJunglerExp, 2)
AverageJunglerExpMin = round((AverageJunglerExp / AverageTime) * 60, 2)
    
# calcul du gain d exp et de gold des adc
CarryGoldEarned = []
CarryExp = []
AverageCarryExp = 0
AverageGoldExp = 0
for i in range(len(GamesInfo)):
    for e in range(10):
        if GamesInfo[i]["info"]["participants"][e]["role"] == "CARRY":
            CarryGoldEarned.append(GamesInfo[i]["info"]["participants"][e]["goldEarned"])
AverageCarryGold = sum(CarryGoldEarned) / len(CarryGoldEarned)
AverageCarryGold = round(AverageCarryGold, 2)
AverageCarryGoldMin = round((AverageCarryGold / AverageTime) * 60, 2)

for i in range(len(GamesInfo)):
    for e in range(10):
        if GamesInfo[i]["info"]["participants"][e]["role"] == "CARRY":
            CarryExp.append(GamesInfo[i]["info"]["participants"][e]["champExperience"])
AverageCarryExp = sum(CarryExp) / len(CarryExp)
AverageCarryExp = round(AverageCarryExp, 2)
AverageCarryExpMin = round((AverageCarryExp / AverageTime) * 60, 2)

# calcul du gain d exp et de gold des mid
MidGoldEarned = []
MidExp = []
AverageMidExp = 0
AverageMidGold = 0
for i in range(len(GamesInfo)):
    for e in range(10):
        if GamesInfo[i]["info"]["participants"][e]["individualPosition"] == "MIDDLE":
            MidGoldEarned.append(GamesInfo[i]["info"]["participants"][e]["goldEarned"])
AverageMidGold = sum(MidGoldEarned) / len(MidGoldEarned)
AverageMidGold = round(AverageMidGold, 2)
AverageMidGoldMin = round((AverageMidGold / AverageTime) * 60, 2)

for i in range(len(GamesInfo)):
    for e in range(10):
        if GamesInfo[i]["info"]["participants"][e]["individualPosition"] == "MIDDLE":
            MidExp.append(GamesInfo[i]["info"]["participants"][e]["champExperience"])
AverageMidExp = sum(MidExp) / len(MidExp)
AverageMidExp = round(AverageMidExp, 2)
AverageMidExpMin = round((AverageMidExp / AverageTime) * 60, 2)

# calcul du gain d exp et de gold des top
TopGoldEarned = []
TopExp = []
AverageTopExp = 0
AverageTopGold = 0
for i in range(len(GamesInfo)):
    for e in range(10):
        if GamesInfo[i]["info"]["participants"][e]["individualPosition"] == "TOP":
            TopGoldEarned.append(GamesInfo[i]["info"]["participants"][e]["goldEarned"])
AverageTopGold = sum(TopGoldEarned) / len(TopGoldEarned)
AverageTopGold = round(AverageTopGold, 2)
AverageTopGoldMin = round((AverageTopGold / AverageTime) * 60, 2)

for i in range(len(GamesInfo)):
    for e in range(10):
        if GamesInfo[i]["info"]["participants"][e]["individualPosition"] == "TOP":
            TopExp.append(GamesInfo[i]["info"]["participants"][e]["champExperience"])
AverageTopExp = sum(TopExp) / len(TopExp)
AverageTopExp = round(AverageTopExp, 2)
AverageTopExpMin = round((AverageTopExp / AverageTime) * 60, 2)

In [89]:
print(AverageTopGoldMin)
print(AverageJunglerGoldMin)
print(AverageMidGoldMin)
print(AverageCarryGoldMin)
print(AverageSupportGoldMin)

427.02
431.84
436.32
479.96
291.51


506.84
